<a href="https://colab.research.google.com/github/PPeachPie/Herst/blob/Herst_graph/Herst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import numpy as np
import math

class New_Analysis:
    def __init__(self, data_y, tau=1, del_tau=1):
        self.data_y = data_y
        self.tau = tau
        self.del_tau = del_tau
        self.res_g = []
        self.x = [i for i in np.arange(tau, len(data_y), del_tau)]
        #[i + 1 for i in range(len(self.data_y))]
        self.new_y = []
        self.a = 0
        self.b = 0
        self.__average_y = 0
        self.__result_tuple = []
        self.analysis()

    def rrange(self, tau, val): # Вычисление размаха для текущего tau
        s = 0
        # ave = sum(val)/tau
        ave = np.average(val)
        print(val)
        min = val[0]-ave
        max = val[0]-ave
        for i in val:
            s += i - ave
            if s < min: min = s
            if s > max: max = s
        return max-min

    def SCO(self, tau, val):
        # medium = sum(val)/tau
        medium = np.average(val)
        s = 0
        for i in val:
            s += (i - medium)**2
        return (s / (tau - 0.99999999))**0.5


    def analysis(self):
        self.RS_process()


        av_y = np.average(self.new_y)
        #вычисление вспомогательных величин
        print(self.x, self.new_y)
        y_ = [self.Fx(i) for i in self.x]
        y_av_y = [i - av_y for i in self.new_y]
        y_av_y_2 = [i**2 for i in y_av_y]
        ei = [self.new_y[i] - y_[i] for i in range(len(y_))]
        ei_2 = [i**2 for i in ei]
        Ai = [abs(1 - (self.new_y[i] / y_[i])) for i in range(len(y_))]
        delta_ei = [ei[i] - ei[i-1] for i in range(len(ei), 2)]
        delta_ei_2 = [i**2 for i in delta_ei]

        #вычисление основных показателей и внесение в результирующий лист с показателями
        r = self.corr_indx(sum(ei_2), sum(y_av_y_2)) #индекс корреляции
        self.__result_tuple.append(r)
        self.__result_tuple.append(r**2) #индекс детерминации
        self.__result_tuple.append(np.average(Ai) * 100) #средняя ошибка детерминации
        k2 = len(self.data_y)-1 #коэфициент необходимый для вычисления критерия Фишера
        self.__result_tuple.append(self.fischer_crit(r, k2)) #критерий фишера
        self.__result_tuple.append(sum(delta_ei_2) / sum(ei_2)) #критерий дарбина-уотсона

    def RS_process(self):
        # self.ls()
        print(self.data_y)
        for i in range(1, len(self.data_y)):
          def_R = self.rrange(i, self.data_y[:i])
          def_S = self.SCO(i, self.data_y[:i])
          if math.isnan(def_R / def_S):
            self.new_y.append(0)
          else:
            self.new_y.append(def_R / def_S)
          self.x.append(i)
          self.res_g.append(self.Fx(i))
        self.new_y = np.array(self.new_y)
        self.ls()
        # self.a = self.LS(self.x, self.new_y)[0]
        # self.b = self.LS(self.x, self.new_y)[1]
        # for i in self.x:
        #     self.res_g.append(self.Fx(i))

    def Fx(self, x):
        return self.a * (x ** self.b)


    def ls(self):
        sX = 0
        sY = 0
        sXY = 0
        sXX = 0
        x = self.x
        y = self.new_y
        n = len(self.new_y)
        print(self.x, self.new_y)
        for i in range(len(y)):
          sX += math.log(x[i])
          sXX += math.log(x[i]) ** 2
          if (y[i] > 0):
            sY += math.log(y[i])
            sXY += math.log(x[i]) * math.log(y[i])
        print(n, sX, sY, sXY, sXX)
        self.b = (n * sXY - sX * sY) / (n * sXX - sX ** 2)
        self.a = math.exp((sY - self.b * sX) / n)
        print(self.a, self.b)

    def corr_indx(self, y1, y2):
        print('here is error')
        print(y1, y2, 1-(y1/y2))
        return math.sqrt(1-(y1/y2))

    def fischer_crit(self, r, k2):
        return r**2 * k2 / (1 - r**2)

    def result_tuple(self):
        return self.__result_tuple


In [49]:
a = New_Analysis([4.4,  3.6,   3,   2.7,  2.1,  1.8,  1.9,  1.5,  1.4,  1.3,  1.2,  1.1,  1.1], 2.5, 0.5)
a.analysis()

[4.4, 3.6, 3, 2.7, 2.1, 1.8, 1.9, 1.5, 1.4, 1.3, 1.2, 1.1, 1.1]
[4.4]
[4.4, 3.6]
[4.4, 3.6, 3]
[4.4, 3.6, 3, 2.7]
[4.4, 3.6, 3, 2.7, 2.1]
[4.4, 3.6, 3, 2.7, 2.1, 1.8]
[4.4, 3.6, 3, 2.7, 2.1, 1.8, 1.9]
[4.4, 3.6, 3, 2.7, 2.1, 1.8, 1.9, 1.5]
[4.4, 3.6, 3, 2.7, 2.1, 1.8, 1.9, 1.5, 1.4]
[4.4, 3.6, 3, 2.7, 2.1, 1.8, 1.9, 1.5, 1.4, 1.3]
[4.4, 3.6, 3, 2.7, 2.1, 1.8, 1.9, 1.5, 1.4, 1.3, 1.2]
[4.4, 3.6, 3, 2.7, 2.1, 1.8, 1.9, 1.5, 1.4, 1.3, 1.2, 1.1]
[2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0, 11.5, 12.0, 12.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] [0.         0.70710678 1.0440738  1.53333334 1.91081893 2.28539309
 2.74826574 3.20114347 3.66977004 4.08578464 4.46071356 4.80174135]
12 19.176040109013382 9.544478875383378 17.826201918423696 32.180993045173835
0.1526189008659532 1.674079444967688
[2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0, 11.5, 12.0, 12.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] [0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:72: RuntimeWarning: invalid value encountered in double_scalars


IndexError: ignored